In [23]:
# for encoding and downloading the file
import base64
from ipywidgets import HTML

# for accessing data from Canvas
from canvasapi import Canvas

# for working with csv.reader
import csv

# pandas data manipulation
import pandas as pd

# for working with tableau
import tableauserverclient as TSC

# create widgets within jupyter notebook
import ipywidgets as widgets
from IPython.display import display

# lets us treat strings as files - used to get
# csv data from tableau to pandas
from io import StringIO

In [2]:
tableauToken = widgets.Password(
    placeholder='Enter Tableau Token',
    description='Tableau Token:',
    disabled=False
)
display(tableauToken)

canvasToken = widgets.Password(
    placeholder='Enter Canvas Token',
    description='Canvas Token:',
    disabled=False
)
display(canvasToken)

Password(description='Tableau Token:', placeholder='Enter Tableau Token')

In [8]:
tableau_auth = TSC.PersonalAccessTokenAuth('Python', tableauToken.value, 'ivytechcommunitycollege')
server = TSC.Server('https://us-east-1.online.tableau.com', use_server_version=True)

In [9]:
CANVAS_API_URL = "https://ivytech.instructure.com"
canvas = Canvas(CANVAS_API_URL, canvasToken.value)

In [10]:
def csv_to_pandas(csv):
    # create an empty string
    mydata = ""
    # for every row in the raw csv file, decode as utf-8 text and append to the empty string
    for row in csv.csv:
        mydata = mydata + row.decode('utf-8')
    # StringIO lets us treat a string as if it was a file, for functions that expect files as input
    p = pd.read_csv(StringIO(mydata))
    return p

In [11]:
# we've hardcoded the id of the view here - explain later.
with server.auth.sign_in(tableau_auth):
    view_id = "3b1377aa-baaa-417a-9006-3b5c5eac403e"
    view = server.views.get_by_id(view_id)
    server.views.populate_csv(view)
    df = csv_to_pandas(view)

In [12]:
# pandas - remove (in-place, modifying existing dataframe) all rows
# in which the value in the given column is not 'Y'
df.drop(df[df['Registered in 2nd-8 Only'] != 'Y'].index, inplace = True)

In [ ]:
# get the original course in which they were erroneously enrolled before
## can be removed after this next batch of runs most likely
# also get the section id of the target course
original_course = 1102530
nosa_course = "placeholder"
course = canvas.get_course(original_course)

# get all student ids for students who are still active in the original course
students = course.get_users(enrollment_types=['Student'])

# fancy python way of building a list
studentids = [x.sis_user_id for x in students]

In [ ]:
# pandas filter - display only students who are *not* in the list of student ids
# of students who were in the original course
df_filtered = df[~df['Student Id'].isin(studentids)]

In [ ]:
# text header row for our enrollments csv - \n means newline
enrollments = ["section_id,user_id,role,status\n"]
# add a line for each student enrollment
for index, row in df_filtered.iterrows():
    enrollments.append(f"{nosa_course},{row['Student Id']},student,active\n")

In [ ]:
# combine the list of enrollments into one big string
txt_enrollments = ''.join(enrollments)

In [ ]:
# stolen from https://stackoverflow.com/questions/61708701/how-to-download-a-file-using-ipywidget-button
# this takes the string list of enrollments from before and displays an html button that'll send the user the
# enrollments.txt we generated.

res = 'computed results'

#FILE
filename = 'enrollments.txt'
b64 = base64.b64encode(txt_enrollments.encode())
payload = b64.decode()

#BUTTONS
html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:text/csv;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
</a>
</body>
</html>
'''

html_button = html_buttons.format(payload=payload,filename=filename)
display(HTML(html_button))